In [ ]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [122]:
def scrape_NFL_REF_QB(player_name):

    first_name = player_name.split(' ')[0].lower()
    last_name = player_name.split(' ')[1].lower()
    player_url = f'https://www.sports-reference.com/cfb/players/{first_name}-{last_name}-1.html'
    html_content = requests.get(player_url).text

    if len(player_name.split(' ')) > 2:
        first_name = player_name.split(' ')[0].lower()
        last_name = player_name.split(' ')[1].lower()
        suffix = player_name.split(' ')[2].lower()
        player_url = f'https://www.sports-reference.com/cfb/players/{first_name}-{last_name}-{suffix}-1.html'
        html_content = requests.get(player_url).text


    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find('table', {'id': 'passing_standard'})

    if not table:
        for idx in range(2, 6):
            time.sleep(3)
            first_name = player_name.split(' ')[0].lower()
            last_name = player_name.split(' ')[1].lower()
            player_url = f'https://www.sports-reference.com/cfb/players/{first_name}-{last_name}-{idx}.html'
            html_content = requests.get(player_url).text
            
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find('table', {'id': 'passing_standard'})
            
            if not table:
                print(f"Passing stats table not found for {player_name}. Failed on index {idx}")
            else:
                print(f"Player found on index {idx}")
                break
    
    try:
        table.find_all('th')
        headers = [th.getText() for th in table.find_all('th')]
        yrs = [i for i in headers if ('2' in i) & (len(i) == 4 or len(i) == 5)]
        career_idx = headers.index(yrs[-1]) + 1
        baseline_headers = []
        headers = [th.getText() for th in table.find_all('th')]
        rows = []

        for tr in table.find_all('tr')[1:]:
            cells = [td.getText() for td in tr.find_all('td')]
            if cells: 
                rows.append(cells)

        career_stats = []

        for row in rows:
            if row.count('') == 2:
                career_stats.extend(row)
                break

        career_stats.remove('')
        career_stats.remove('')

        career_stats.append(len(yrs))
        career_stats.append(player_name)

        column_names = [
        'G',        
        'Cmp',      
        'Att',      
        'Cmp%',     
        'Yds',      
        'TD',       
        'TD%',      
        'Int',      
        'Int%',     
        'Y/A',      
        'AY/A',     
        'Y/C',      
        'Y/G',      
        'Rate',
        'seasons',
        'name'
        ]

        final = pd.DataFrame({name: [value] for name, value in zip(column_names, career_stats)})

        if len(career_stats) == len(column_names):
            final = pd.DataFrame({name: [value] for name, value in zip(column_names, career_stats)})
        else:
            print("Error: Number of stats does not match number of column names.")

        return final    
    except:
        print(f"FAILED: On {player_name}")

In [127]:
tb = scrape_NFL_REF_QB('Russell Wilson')

In [128]:
tb

,G,Cmp,Att,Cmp%,Yds,TD,TD%,Int,Int%,Y/A,AY/A,Y/C,Y/G,Rate,seasons,name
0,50,907,1489,60.9,11720,109,7.3,30,2.0,7.9,8.43,12.9,234.4,147.2,4,Russell Wilson


In [100]:
tb.find_all('th')
headers = [th.getText() for th in tb.find_all('th')]
yrs = [i for i in headers if ('2' in i) & (len(i) == 4 or len(i) == 5)]
career_idx = headers.index(yrs[-1]) + 1
baseline_headers = []
headers = [th.getText() for th in tb.find_all('th')]
rows = []

for tr in tb.find_all('tr')[1:]:
    cells = [td.getText() for td in tr.find_all('td')]
    if cells: 
        rows.append(cells)

to_app = []

for row in rows:
    if row.count('') == 2:
        to_app.extend(row)
        break

to_app.remove('')
to_app.remove('')

to_app.append(len(yrs))
to_app.append(player_name)

column_names = [
'G',        
'Cmp',      
'Att',      
'Cmp%',     
'Yds',      
'TD',       
'TD%',      
'Int',      
'Int%',     
'Y/A',      
'AY/A',     
'Y/C',      
'Y/G',      
'Rate',
'seasons',
'name'
]

final = pd.DataFrame({name: [value] for name, value in zip(column_names, to_app)})

[<th aria-label="Season" class="poptip center" data-stat="year_id" scope="col">Season</th>,
 <th aria-label="Team" class="poptip sort_default_asc center" data-stat="team_name_abbr" scope="col">Team</th>,
 <th aria-label="Conf" class="poptip sort_default_asc center" data-stat="conf_abbr" data-tip="Conference" scope="col">Conf</th>,
 <th aria-label="Class" class="poptip center" data-stat="class" data-tip="Class" scope="col">Class</th>,
 <th aria-label="Pos" class="poptip center" data-stat="pos" data-tip="Position" scope="col">Pos</th>,
 <th aria-label="G" class="poptip center" data-stat="games" data-tip="&lt;strong&gt;Games played&lt;/strong&gt;&lt;br /&gt;Includes bowl games since 2002" scope="col">G</th>,
 <th aria-label="Cmp" class="poptip center" data-stat="pass_cmp" data-tip="Passes completed" scope="col">Cmp</th>,
 <th aria-label="Att" class="poptip center" data-stat="pass_att" data-tip="Passes attempted" scope="col">Att</th>,
 <th aria-label="Cmp%" class="poptip hide_non_quals cen

In [107]:
baseline_headers = []
headers = [th.getText() for th in tb.find_all('th')]
rows = []
for tr in tb.find_all('tr')[1:]:
    cells = [td.getText() for td in tr.find_all('td')]
    if cells: 
        rows.append(cells)

In [108]:
to_app = []
for row in rows:
    if row.count('') == 2:
        to_app.extend(row)
        break

In [111]:
to_app.remove('')

In [118]:
to_app.append(len(yrs))
to_app.append('Trevor Lawrence')

In [119]:
column_names = [
'G',        
'Cmp',      
'Att',      
'Cmp%',     
'Yds',      
'TD',       
'TD%',      
'Int',      
'Int%',     
'Y/A',      
'AY/A',     
'Y/C',      
'Y/G',      
'Rate',
'seasons',
'name'
]

In [120]:
final = pd.DataFrame({name: [value] for name, value in zip(column_names, to_app)})

In [121]:
final

,G,Cmp,Att,Cmp%,Yds,TD,TD%,Int,Int%,Y/A,AY/A,Y/C,Y/G,Rate,seasons,name
0,40,758,1138,66.6,10098,90,7.9,17,1.5,8.9,9.78,13.3,252.5,164.3,3,Trevor Lawrence


In [ ]:
yr_college = len(rows) - 1

column_names = [
'G',        
'Cmp',      
'Att',      
'Cmp%',     
'Yds',      
'TD',       
'TD%',      
'Int',      
'Int%',     
'Y/A',      
'AY/A',     
'Y/C',      
'Y/G',      
'Rate',
'seasons',
'name'
]

totals = rows[len(rows) - 2]
totals = [stat for stat in totals if stat.strip()]
totals.append(yr_college)
totals.append(player_name)